# Model Registry

In [ ]:
import mlflow
from mlflow import MlflowClient
import os
from dotenv import load_dotenv
load_dotenv()

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(os.getenv("MLFLOW_EXPERIMENT"))

## Register a model to Model Registry

In [ ]:
"""
mlflow.<model_flavor>.log_model(registered_model_name=<model_name>) # register the model while logging it to the tracking server.
mlflow.register_model(<model_uri>, <model_name>) # register the model after logging it to the tracking server. Note that you'll have to log the model before running this command to get a model URI.
"""

## Load a Registered Model - To perform inference on a registered model version

In [ ]:
# Use model ID
model_id = 'm-5793c91765c748d989118aa7e93e6bd8'
logged_model = mlflow.get_logged_model(model_id)

print(logged_model, logged_model.metrics, logged_model.params)

In [ ]:
# Use run ID
# model = mlflow.sklearn.load_model(f"runs:/{mlflow_run_id}/{run_relative_path_to_model}")


In [ ]:
# Use registered model name, and version
model_uri = "models:/logistic-regression-model-a/3" #"models:/{registered_model_name}/{model_version}"
model = mlflow.sklearn.load_model(model_uri)

print(model)

In [ ]:
# Load via Model Version Alias
# Reassign alias from the old model version to the new one
client = MlflowClient()

# Set model version alias
model_name = "logistic-regression-model-a"
model_version_alias = "challenger"

model_info = client.get_model_version_by_alias(model_name, model_version_alias)
model_tags = model_info.tags
print(model_tags)

# Get the model version using a model URI
model_uri = f"models:/{model_name}@{model_version_alias}"
model = mlflow.sklearn.load_model(model_uri)

print(model)

## Search Models

In [ ]:
# Searching By Metrics

experimend_id = '254433368725384354'


# Find high-performing models
high_accuracy_models = mlflow.search_logged_models(
    experiment_ids=[experimend_id],  # Replace with your experiment ID
    filter_string="metrics.test_accuracy > 0.9",
)
print(high_accuracy_models)

In [ ]:
from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

## Deploy and Organize Models with Aliases and Tags

In [ ]:
client = MlflowClient()

client.set_registered_model_alias(
    model_name, model_version_alias, "3" #sets "the_best_model_ever" to the version 1 model.
)  # or do it manually in UI

In [ ]:
client = MlflowClient()

client.set_registered_model_alias(
    "creditcard_model_staging", "candidate", "2" #sets "the_best_model_ever" to the version 1 model.
) 

In [ ]:
# get a model version by alias
client.get_model_version_by_alias("creditcard_model_prod", "Champion")

# delete the alias
client.delete_registered_model_alias("creditcard_model_prod", "Sample")

In [16]:
client = MlflowClient()

ALGORITHM_TYPE = "logistic-regression"
REGISTERED_MODEL_NAME = "fraud-detection-model"

client.set_model_version_tag(
    REGISTERED_MODEL_NAME,
    '9',
    "algorithm",
    ALGORITHM_TYPE
)

In [ ]:
for i in range(1,9):
    client.set_model_version_tag(
        REGISTERED_MODEL_NAME,
        str(i),
        "algorithm",
        ALGORITHM_TYPE
    )

## Promoting an MLflow Model across environments

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
client.copy_model_version(
    src_model_uri="models:/creditcard_model_staging@candidate",
    dst_name="creditcard_model_prod",
)

## Renaming an MLflow Model

In [ ]:
client = MlflowClient()
client.rename_registered_model(
    name="logistic-regression-model",
    new_name="logistic-regression-model-d",
)